In [26]:
import numpy as np
import random

# Parameters
NUM_EXAMS = 10
NUM_TIMESLOTS = 5
NUM_ROOMS = 3
ROOM_CAPACITY = [30, 50, 40]  # Room capacities
NUM_PROFESSORS = NUM_EXAMS  # One professor per exam
NUM_ANTS = 20
NUM_ITERATIONS = 100
PHEROMONE_INIT = 1.0
EVAPORATION_RATE = 0.5
ALPHA = 1.0
BETA = 2.0

# Input data
STUDENT_EXAM_MATRIX = np.random.randint(0, 2, (50, NUM_EXAMS))  # Students in exams
PROFESSOR_AVAILABILITY = np.random.randint(0, 2, (NUM_PROFESSORS, NUM_TIMESLOTS))  # Availability
PREFERRED_TIMES = {0: [0, 1], 1: [2], 2: [1, 3]}  # Preferred timeslots for professors
ROOM_ADJACENCY = np.random.rand(NUM_ROOMS, NUM_ROOMS)  # Closer rooms have higher weights

# Initialize pheromone matrix
pheromone = np.ones((NUM_EXAMS, NUM_TIMESLOTS, NUM_ROOMS)) * PHEROMONE_INIT

def calculate_heuristic_matrix():
    heuristic = np.zeros((NUM_EXAMS, NUM_TIMESLOTS, NUM_ROOMS))
    for exam in range(NUM_EXAMS):
        for timeslot in range(NUM_TIMESLOTS):
            for room in range(NUM_ROOMS):
                # Check professor availability
                professor_availability = PROFESSOR_AVAILABILITY[exam, timeslot]
                
                # Room capacity
                capacity_factor = ROOM_CAPACITY[room]
                
                # Conflict penalty
                conflict_penalty = np.sum([
                    STUDENT_EXAM_MATRIX[student, exam] * STUDENT_EXAM_MATRIX[student, other_exam]
                    for student in range(STUDENT_EXAM_MATRIX.shape[0])
                    for other_exam in range(NUM_EXAMS) if other_exam != exam
                ])
                
                # Preference bonus
                preference_bonus = 1 if timeslot in PREFERRED_TIMES.get(exam, []) else 0.5
                
                heuristic[exam, timeslot, room] = (
                    professor_availability * preference_bonus * capacity_factor / (1 + conflict_penalty)
                )
    return heuristic


# Fitness function
def evaluate_solution(solution):
    penalty = 0
    
    for student in range(STUDENT_EXAM_MATRIX.shape[0]):
        exams = np.where(STUDENT_EXAM_MATRIX[student] == 1)[0]
        timeslots = [solution[exam][0] for exam in exams]
        penalty += len(timeslots) - len(set(timeslots))  # Overlap penalty
    
    for exam, (timeslot, room) in solution.items():
        if not PROFESSOR_AVAILABILITY[exam, timeslot]:  # Professor unavailable
            penalty += 10
        if timeslot not in PREFERRED_TIMES.get(exam, []):  # Non-preferred timeslot
            penalty += 2
    
    return penalty

# ACO Algorithm
def aco_timetabling():
    global pheromone
    best_solution = None
    best_fitness = float("inf")

    for iteration in range(NUM_ITERATIONS):
        solutions = []
        fitness_scores = []

        # Ants construct solutions
        for ant in range(NUM_ANTS):
            solution = {}
            heuristic = calculate_heuristic_matrix()

            for exam in range(NUM_EXAMS):
                probabilities = np.zeros((NUM_TIMESLOTS, NUM_ROOMS))
                for timeslot in range(NUM_TIMESLOTS):
                    for room in range(NUM_ROOMS):
                        probabilities[timeslot, room] = (
                            pheromone[exam, timeslot, room] ** ALPHA
                            * heuristic[exam, timeslot, room] ** BETA
                        )

                probabilities /= probabilities.sum()
                chosen_slot_room = np.unravel_index(
                    np.random.choice(range(probabilities.size), p=probabilities.flatten()),
                    probabilities.shape,
                )
                solution[exam] = chosen_slot_room

            fitness = evaluate_solution(solution)
            solutions.append(solution)
            fitness_scores.append(fitness)

        # Update pheromones
        pheromone *= (1 - EVAPORATION_RATE)
        for i, solution in enumerate(solutions):
            for exam, (timeslot, room) in solution.items():
                pheromone[exam, timeslot, room] += 1.0 / fitness_scores[i]

        # Track the best solution
        min_fitness = min(fitness_scores)
        if min_fitness < best_fitness:
            best_fitness = min_fitness
            best_solution = solutions[fitness_scores.index(min_fitness)]

        print(f"Iteration {iteration + 1}: Best Fitness = {best_fitness}")

    return best_solution, best_fitness

# Run the ACO algorithm
best_solution, best_fitness = aco_timetabling()
print("Best Solution:", best_solution)
print("Best Fitness:", best_fitness)


Iteration 1: Best Fitness = 80
Iteration 2: Best Fitness = 80
Iteration 3: Best Fitness = 80
Iteration 4: Best Fitness = 80
Iteration 5: Best Fitness = 80
Iteration 6: Best Fitness = 80
Iteration 7: Best Fitness = 80
Iteration 8: Best Fitness = 80
Iteration 9: Best Fitness = 80
Iteration 10: Best Fitness = 80
Iteration 11: Best Fitness = 80
Iteration 12: Best Fitness = 80
Iteration 13: Best Fitness = 80
Iteration 14: Best Fitness = 80
Iteration 15: Best Fitness = 80
Iteration 16: Best Fitness = 80
Iteration 17: Best Fitness = 80
Iteration 18: Best Fitness = 80
Iteration 19: Best Fitness = 80
Iteration 20: Best Fitness = 80
Iteration 21: Best Fitness = 80
Iteration 22: Best Fitness = 80
Iteration 23: Best Fitness = 80
Iteration 24: Best Fitness = 80
Iteration 25: Best Fitness = 80
Iteration 26: Best Fitness = 80
Iteration 27: Best Fitness = 80
Iteration 28: Best Fitness = 80
Iteration 29: Best Fitness = 80
Iteration 30: Best Fitness = 80
Iteration 31: Best Fitness = 80
Iteration 32: Bes

In [34]:
import numpy as np
import random

# Problem data
exams = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J']
timeslots = ['9:00 AM to 11:00 AM', '12:00 PM to 2:00 PM']
exam_halls = ['Exam Hall 1', 'Exam Hall 2', 'Exam Hall 3']
examiners = ['Examiner 1', 'Examiner 2', 'Examiner 3', 'Examiner 4', 'Examiner 5']
students = 100

# Parameters for ACO
num_ants = 10
num_iterations = 100
alpha = 1  # Pheromone importance
beta = 2   # Heuristic importance
evaporation_rate = 0.5
pheromone_constant = 100

epsilon = 1e-6  # Small constant to prevent division by zero

# Initialize pheromone matrix and heuristic information
pheromone = np.ones((len(exams), len(timeslots), len(exam_halls), len(examiners)))
heuristic = np.random.rand(len(exams), len(timeslots), len(exam_halls), len(examiners))

# Helper function to calculate solution cost (lower is better)
def calculate_cost(solution):
    cost = 0
    assigned = set()
    for exam, (timeslot, exam_hall, examiner) in solution.items():
        if (timeslot, exam_hall, examiner) in assigned:
            cost += 10  # Penalize conflicts
        assigned.add((timeslot, exam_hall, examiner))
    return cost

# Ant Colony Optimization algorithm
def ant_colony_optimization():
    global pheromone
    best_solution = None
    best_cost = float('inf')

    for iteration in range(num_iterations):
        solutions = []
        costs = []

        for ant in range(num_ants):
            solution = {}
            for i, exam in enumerate(exams):
                probabilities = []
                for t in range(len(timeslots)):
                    for r in range(len(exam_halls)):
                        for e in range(len(examiners)):
                            prob = (pheromone[i][t][r][e] ** alpha) * (heuristic[i][t][r][e] ** beta)
                            probabilities.append((prob, t, r, e))
                probabilities = sorted(probabilities, key=lambda x: x[0], reverse=True)
                chosen = random.choices(probabilities, weights=[p[0] for p in probabilities], k=1)[0]
                solution[exam] = (timeslots[chosen[1]], exam_halls[chosen[2]], examiners[chosen[3]])

            cost = calculate_cost(solution)
            solutions.append(solution)
            costs.append(cost)

        # Update pheromones
        pheromone *= (1 - evaporation_rate)
        for solution, cost in zip(solutions, costs):
            for exam, (timeslot, exam_hall, examiner) in solution.items():
                i = exams.index(exam)
                t = timeslots.index(timeslot)
                r = exam_halls.index(exam_hall)
                e = examiners.index(examiner)
                pheromone[i][t][r][e] += pheromone_constant / (cost + epsilon)

        # Track the best solution
        min_cost_idx = np.argmin(costs)
        if costs[min_cost_idx] < best_cost:
            best_cost = costs[min_cost_idx]
            best_solution = solutions[min_cost_idx]

    return best_solution

# Run the algorithm
best_timetable = ant_colony_optimization()

# Display the result
def display_timetable(timetable):
    print("Exam\t--Timeslot--\t\t--Exam Hall--\t--Examiner--")
    for exam, (timeslot, exam_hall, examiner) in timetable.items():
        print(f"{exam}\t{timeslot}\t{exam_hall}\t{examiner}")

display_timetable(best_timetable)


Exam	--Timeslot--		--Exam Hall--	--Examiner--
A	12:00 PM to 2:00 PM	Exam Hall 1	Examiner 5
B	12:00 PM to 2:00 PM	Exam Hall 2	Examiner 3
C	12:00 PM to 2:00 PM	Exam Hall 3	Examiner 5
D	9:00 AM to 11:00 AM	Exam Hall 3	Examiner 4
E	12:00 PM to 2:00 PM	Exam Hall 1	Examiner 3
F	9:00 AM to 11:00 AM	Exam Hall 3	Examiner 3
G	9:00 AM to 11:00 AM	Exam Hall 3	Examiner 5
H	12:00 PM to 2:00 PM	Exam Hall 2	Examiner 5
I	9:00 AM to 11:00 AM	Exam Hall 2	Examiner 1
J	9:00 AM to 11:00 AM	Exam Hall 3	Examiner 1
